<a href="https://colab.research.google.com/github/saktheeswaranswan/random-math-addition-data-for-rnn-generator/blob/main/3d_fem_gst_assembly_full_complete_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import pandas as pd

def assemble_element_stiffness(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4, youngs_modulus, poisson_ratio, thickness):
    # Calculate the element stiffness matrix for a single element
    # using the coordinates of the four nodes and material properties
    a = np.array([x2-x1, y2-y1, z2-z1])
    b = np.array([x3-x1, y3-y1, z3-z1])
    c = np.array([x4-x1, y4-y1, z4-z1])
    area = np.linalg.norm(np.cross(b, c))/2
    l = np.linalg.norm(a)
    e = youngs_modulus
    v = poisson_ratio
    ke = np.array([[1, v, 0, 0, 0, 0],
                   [v, 1, 0, 0, 0, 0],
                   [0, 0, (1-v)/2, 0, 0, 0],
                   [0, 0, 0, (1-v)/2, 0, 0],
                   [0, 0, 0, 0, (1-v)/2, 0],
                   [0, 0, 0, 0, 0, (1-v)/2]])
    ke = e/(1-v**2)*np.array([[a[0]**2, a[0]*a[1], a[0]*a[2], a[1]*a[0], a[1]**2, a[1]*a[2]],
                             [a[0]*a[1], a[1]**2, a[1]*a[2], a[0]**2, a[0]*a[1], a[0]*a[2]],
                             [a[0]*a[2], a[1]*a[2], a[2]**2, a[0]*a[2], a[1]*a[2], a[2]**2],
                             [a[1]*a[0], a[0]**2, a[0]*a[2], a[1]**2, a[1]*a[1], a[1]*a[2]],
                             [a[1]**2, a[0]*a[1], a[0]*a[2], a[1]*a[0], a[0]**2, a[0]*a[2]],
                             [a[1]*a[2], a[0]*a[2], a[2]**2, a[1]*a[2], a[0]*a[2], a[0]**2]])
    ke = ke*area*thickness/l
    return ke

def assemble_global_stiffness(element_stiffness_list, n_nodes, n_elements):
    # Assemble the global stiffness matrix from the element stiffness matrices
    global_stiffness = sparse.dok_matrix((3*n_nodes, 3*n_nodes))
    for i in range(n_elements):
        ke = element_stiffness_list[i]
        node_indices = np.array([3*i, 3*i+1, 3*i+2, 3*i+3, 3*i+4, 3*i+5])
        for j in range(6):
            for k in range(6):
                global_stiffness[node_indices[j], node_indices[k]] += ke[j, k]
    return global_stiffness.tocsr()

def plot_sparsity(matrix):
    # Plot the sparsity pattern of the matrix
    plt.spy(matrix, marker='.', markersize=1)
    plt.show()

def solve_displacement(global_stiffness, force_vector):
    # Solve for the displacement using the global stiffness matrix and force vector
    displacement = np.linalg.solve(global_stiffness.toarray(), force_vector)
    return displacement

# Load the coordinates of the nodes and the element connectivity from a CSV file
node_data = pd.read_csv("node_data.csv")
element_data = pd.read_csv("element_data.csv")

n_nodes = node_data.shape[0]
n_elements = element_data.shape[0]

# Extract the x, y, and z coordinates of the nodes
x = node_data["x"].values
y = node_data["y"].values
z = node_data["z"].values

# Extract the indices of the nodes that make up each element
node1 = element_data["node1"].values - 1
node2 = element_data["node2"].values - 1
node3 = element_data["node3"].values - 1
node4 = element_data["node4"].values - 1

# Define the material properties
youngs_modulus = 1.0
poisson_ratio = 0.3
thickness = 1.0

# Calculate the element stiffness matrices for each element
element_stiffness_list = []
for i in range(n_elements):
    x1, y1, z1 = x[node1[i]], y[node1[i]], z[node1[i]]
    x2, y2, z2 = x[node2[i]], y[node2[i]], z[node2[i]]
    x3, y3, z3 = x[node3[i]], y[node3[i]], z[node3[i]]
    x4, y4, z4 = x[node4[i]], y[node4[i]], z[node4[i]]
    ke = assemble_element_stiffness(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4, youngs_modulus, poisson_ratio, thickness)
    element_stiffness_list.append(ke)

# Assemble the global stiffness matrix
global_stiffness = assemble_global_stiffness(element_stiffness_list, n_nodes, n_elements)

# Plot the sparsity pattern of the global stiffness matrix
plot_sparsity(global_stiffness)

# Load the degrees of freedom (DOFs) and the forces from a CSV file
dof_data = pd.read_csv("dof_data.csv")
n_dofs = dof_data.shape[0]

# Extract the DOF indices and the forces
dof_indices = dof_data["dof_index"].values - 1
forces = dof_data["force"].values

# Assemble the force vector
force_vector = np.zeros(3*n_nodes)
for i in range(n_dofs):
    force_vector[dof_indices[i]] = forces[i]

# Solve for the displacement
displacement = solve_displacement(global_stiffness, force_vector)




In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

# Read nodal coordinates and degrees of freedom from a CSV file
nodal_data = np.genfromtxt('nodal_data.csv', delimiter=',')
x = nodal_data[:, 1]
y = nodal_data[:, 2]
z = nodal_data[:, 3]
dof = nodal_data[:, 4:].astype(int)

# Read element connectivity data from a CSV file
element_data = np.genfromtxt('element_data.csv', delimiter=',')
elements = element_data[:, 1:].astype(int)

# Define element stiffness matrix
E = 1.0  # Young's modulus
A = 1.0  # Cross-sectional area
L = 1.0  # Length of the element
k = E * A / L
ke = np.array([[1, -1], [-1, 1]]) * k

# Assemble the global stiffness matrix
n_nodes = x.shape[0]
n_dof = dof.shape[1]
K = np.zeros((n_nodes * n_dof, n_nodes * n_dof))

for element in elements:
    node1, node2 = element
    dof1 = dof[node1 - 1, :]
    dof2 = dof[node2 - 1, :]
    idx = np.hstack([dof1, dof2])
    K[np.ix_(idx, idx)] += ke

# Solve for the displacement
F = np.zeros(n_nodes * n_dof)
U = np.linalg.solve(K, F)
U = tf.convert_to_tensor(U)

# Plot the contribution of element stiffness matrix to the global stiffness matrix
plt.spy(csr_matrix(K), markersize=1)
plt.show()


OSError: ignored

In [ ]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import pandas as pd

def assemble_element_stiffness(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4, youngs_modulus, poisson_ratio, thickness):
    # Calculate the element stiffness matrix for a single element
    # using the coordinates of the four nodes and material properties
    a = np.array([x2-x1, y2-y1, z2-z1])
    b = np.array([x3-x1, y3-y1, z3-z1])
    c = np.array([x4-x1, y4-y1, z4-z1])
    area = np.linalg.norm(np.cross(b, c))/2
    l = np.linalg.norm(a)
    e = youngs_modulus
    v = poisson_ratio
    ke = np.array([[1, v, 0, 0, 0, 0],
                   [v, 1, 0, 0, 0, 0],
                   [0, 0, (1-v)/2, 0, 0, 0],
                   [0, 0, 0, (1-v)/2, 0, 0],
                   [0, 0, 0, 0, (1-v)/2, 0],
                   [0, 0, 0, 0, 0, (1-v)/2]])
    ke = e/(1-v**2)*np.array([[a[0]**2, a[0]*a[1], a[0]*a[2], a[1]*a[0], a[1]**2, a[1]*a[2]],
                             [a[0]*a[1], a[1]**2, a[1]*a[2], a[0]**2, a[0]*a[1], a[0]*a[2]],
                             [a[0]*a[2], a[1]*a[2], a[2]**2, a[0]*a[2], a[1]*a[2], a[2]**2],
                             [a[1]*a[0], a[0]**2, a[0]*a[2], a[1]**2, a[1]*a[1], a[1]*a[2]],
                             [a[1]**2, a[0]*a[1], a[0]*a[2], a[1]*a[0], a[0]**2, a[0]*a[2]],
                             [a[1]*a[2], a[0]*a[2], a[2]**2, a[1]*a[2], a[0]*a[2], a[0]**2]])
    ke = ke*area*thickness/l
    return ke

def assemble_global_stiffness(element_stiffness_list, n_nodes, n_elements):
    # Assemble the global stiffness matrix from the element stiffness matrices
    global_stiffness = sparse.dok_matrix((3*n_nodes, 3*n_nodes))
    for i in range(n_elements):
        ke = element_stiffness_list[i]
        node_indices = np.array([3*i, 3*i+1, 3*i+2, 3*i+3, 3*i+4, 3*i+5])
        for j in range(6):
            for k in range(6):
                global_stiffness[node_indices[j], node_indices[k]] += ke[j, k]
    return global_stiffness.tocsr()

def plot_sparsity(matrix):
    # Plot the sparsity pattern of the matrix
    plt.spy(matrix, marker='.', markersize=1)
    plt.show()

def solve_displacement(global_stiffness, force_vector):
    # Solve for the displacement using the global stiffness matrix and force vector
    displacement = np.linalg.solve(global_stiffness.toarray(), force_vector)
    return displacement

# Load the coordinates of the nodes and the element connectivity from a CSV file
node_data = pd.read_csv("node_data.csv")
element_data = pd.read_csv("element_data.csv")

n_nodes = node_data.shape[0]
n_elements = element_data.shape[0]

# Extract the x, y, and z coordinates of the nodes
x = node_data["x"].values
y = node_data["y"].values
z = node_data["z"].values

# Extract the indices of the nodes that make up each element
node1 = element_data["node1"].values - 1
node2 = element_data["node2"].values - 1
node3 = element_data["node3"].values - 1
node4 = element_data["node4"].values - 1

# Define the material properties
youngs_modulus = 1.0
poisson_ratio = 0.3
thickness = 1.0

# Calculate the element stiffness matrices for each element
element_stiffness_list = []
for i in range(n_elements):
    x1, y1, z1 = x[node1[i]], y[node1[i]], z[node1[i]]
    x2, y2, z2 = x[node2[i]], y[node2[i]], z[node2[i]]
    x3, y3, z3 = x[node3[i]], y[node3[i]], z[node3[i]]
    x4, y4, z4 = x[node4[i]], y[node4[i]], z[node4[i]]
    ke = assemble_element_stiffness(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4, youngs_modulus, poisson_ratio, thickness)
    element_stiffness_list.append(ke)

# Assemble the global stiffness matrix
global_stiffness = assemble_global_stiffness(element_stiffness_list, n_nodes, n_elements)

# Plot the sparsity pattern of the global stiffness matrix
plot_sparsity(global_stiffness)

# Load the degrees of freedom (DOFs) and the forces from a CSV file
dof_data = pd.read_csv("dof_data.csv")
n_dofs = dof_data.shape[0]

# Extract the DOF indices and the forces
dof_indices = dof_data["dof_index"].values - 1
forces = dof_data["force"].values

# Assemble the force vector
force_vector = np.zeros(3*n_nodes)
for i in range(n_dofs):
    force_vector[dof_indices[i]] = forces[i]

# Solve for the displacement
displacement = solve_displacement(global_stiffness, force_vector)


